# Chap3. word2vec
## 3.2 단순한 word2vec

앞 절(3.1 추론 기반 기법과 신경망)에서는 추론 기반 기법을 배우고, 신경망으로 단어를 처리하는 방법을 코드로 살펴보았다.<br>
word2vec을 구현해보자!

지금부터 할 일은 모델을 신경망으로 구축하는 것이다.<br>
이번절에서 사용할 신경망은 word2vec에서 제안하는 <code>CBOW(continuous bag-of-words)모델</code>이다.

*Note*<br>
<code>CBOW(Continuous Bag-Of-Words) 모델</code>과 <code>skip-gram 모델</code>은 word2vec에서 사용되는 신경망이다.<br>
이번 절에서는 CBOW 모델을 중심으로 이야기를 풀어갈 것이다. 두 모델의 차이는 '3.5.2 skip-gram 모델' 절에서 자세히 설명한다.

### 3.2.1 CBOW 모델의 추론 처리 

CBOW 모델은 맥락으로부터 타겟(target)을 추측하는 용도의 신경망이다.<br>
여기서 '타겟'이란 중앙단어고 '맥락'은 주변 단어이다.<br>
우리는 이 CBOW모델이 가능한 한 정확하게 추론하도록 훈련시켜서 단어의 분산 표현을 얻어낼 것이다.

* CBOW 모델의 입력과 출력
    * CBOW 모델의 입력은 맥락(context)이다.
    * 맥락을 one-hot vector로 표현하여 CBOW 모델이 처리할 수 있도록 준비한다.

* CBOW 모델의 신경망
    * 두 입력층(모델에 따라 입력층의 개수는 상이)에서 은닉층으로의 변환은 똑같은 완전연결계층(가중치는 Win)이 처리한다.
    * 은닉층에서 출력층 뉴런으로의 변환은 다른 완전연결계층(가중치는 Wout)이 처리한다.

* 입력층의 개수는 어떻게 구하는 걸까?
    * **맥락으로 고려할 단어**를 2개로 정했다면 입력층은 2개이다.
    * 즉, 맥락에 포함시킬 단어가 N개라면 입력층도 N개가 된다.(맥락으로 고려할 단어의 수가 입력층의 개수가 된다)

* 은닉층에 주목하자
    * 은닉층의 뉴런은 입력층의 완전연결계층에 의해 변환된 값이다.
    * 입력층이 여러 개이면 전체를 '평균'하면 된다.
        * 맥락으로 고려할 단어가 2개라면 전체 입력층(2개)이 완전연결 계층을 통과한 값을 2로 나눠주면 되겠군!

* 마지막으로 출력층에 주목을 하자
    * 출력층의 뉴런 하나하나는 각각의 단어에 대응한다
    * 출력 뉴런은 각 단어의 '점수'를 뜻하며 값이 높을수록 대응 단어의 출현 확률도 높아진다
    * 여기서 점수란 확률로 해석되기 전의 값이고 확률로 표현하기 위해서는 '소프트맥스'함수를 적용하면 된다.

입력층에서 은닉층으로의 변환은 완전연결계층(가중치는 Win)에 의해서 이루어진다.<br>
미리 밝히자면 이 가중치가 바로 단어의 분산 표현의 정체이다.<br>

Win의 각 행에는 해당 단어의 분산 표현이 담겨 있다. 따라서 학습을 진행할수록 맥락에서 출현하는 단어를 잘 추측하는 방향으로 이 분산 표현들이 갱신될 것이다. <br>
또한 놀랍게도 이렇게 해서 얻은 벡터에는 '단어의 의미'도 잘 녹아들어 있다! 이것이 바로 word2vec의 전체 그림이다.

*Note*<br>
은닉층의 뉴런 수를 입력층의 뉴런 수보다 적게 하는 것이 중요한 핵심이다.<br>
이렇게 해야 은닉층에는 단어 예측에 필요한 정보를 '간결하게 담게 되며, 결과적으로 밀집벡터 표현을 얻을 수 있다.<br>
이 때 은닉층의 정보는 우리 인간이 이해할 수 없는 코드로 쓰여져있다. 바로 '인코딩'에 해당되기 때문이다.<br>
한편 은닉층의 정보로부터 원하는 결과를 얻는 작업은 '디코딩'이라고 한다.<br>
즉, 디코딩이란 인코딩된 정보를 우리 인간이 이해할 수 있는 표현으로 복원하는 작업이다.

* 파이썬 구현

In [4]:
# 책에 있는 text CBOW모델 구현
# 'You say goodbye I say hello.'
# text는 전처리를 했으며, 각각의 단어를 원-핫 벡터로 표현한다.

import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

# 샘플 맥락 데이터
# 맥락이 되는 단어의 수를 2개로 정한다
# 따라서 입력 레이어의 수는 2개이다.
c0 = np.array([1, 0, 0, 0, 0, 0, 0])
c1 = np.array([0, 0, 1, 0, 0, 0, 0])

# 1. 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

# 2. 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 3. 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 1/2 * (h0 + h1)  # 은닉층의 뉴런: 입력층을 통과한 레이어의 평균을 구해준다.
s = out_layer.forward(h)

print(s)

[ 0.32343283  0.04135157 -0.00599753  0.65007823 -0.44451717 -0.27830561
  0.35841084]


CBOW 모델은 활성화 함수를 사용하지 않는 간단한 구성의 신경망이다.<br>
입력층이 여러 개 있고, 그 입력층들이 가중치를 공유한다는 점을 제외하면 어려운 부분은 없다.

### 3.2.2 CBOW 모델의 학습

지금까지 구현한 CBOW모델은 출력층에서 각 단어의 점수를 출력했다. 이 점수에 소프트맥스 함수를 적용하면 '확률'을 얻을 수 있다.<br>
이 확률은 맥락이 주어졌을 때 그 중앙에 어떤 단어가 출현하는지를 나타낸다.<br>


'가중치가 적절히 설정된' 신경망이라면 '확률'을 나타내는 뉴런들 중 정답에 해당하는 뉴런의 값이 클 것이라 기대할 수 있다.

* 단어의 출현 패턴을 파악한 벡터 학습
    * CBOW모델(과 skip-gram 모델)로 얻을 수 있는 단어의 분산 표현은 단어의 의미 면에서나 문법 면에서 모두 우리의 직관에 부합하는 경우를 많이 볼 수 있다.

*NOTE*<br>
CBOW모델은 단어 출현 패턴을 학습 시 사용한 말뭉치로부터 배운다. 따라서 말뭉치가 다르면 학습 후 얻게 되는 단어의 분산 표현도 달라진다.<br>
예를 들어 말뭉치로 '스포츠'기사만 사용하는 경우와 '음악'관련 기사만 사용하는 경우 얻게되는 단어의 분산 표현이 크게 다르다.

* 다중 클래스 분류 신경망
    * 다중 클래스 분류를 수행하는 신경망을 학습하려면 소프트맥스와 교차 엔트로피 오차를 이용하면 된다.
    * 소프트맥스 함수를 이용해 점수를 확률로 변환하고, 확률과 정답 레이블로부터 교차 엔트로피 오차를 구한다.
    * 교차 엔트로피의 출력 값을 '손실'로 사용해 신경망의 학습을 수행한다.

<code>다중 클래스 분류 신경망</code>구현을 위해 앞서 구현한 CBOW 모델에 Softmax 계층과 Cross Entropy Error 계층을 추가해준다.<br>
이상이 CBOW 모델의 손실을 구하는 계산의 흐름이자, 신경망의 순방향 전파이다.

### 3.2.3 word2vec의 가중치와 분산 표현

* word2vec에서 사용되는 신명망의 두 가지 가중치
    * W-in과 W-out
    * W-in의 각 행은 각 단어의 분산 표현에 해당한다.
    * W-out에도 단어의 의미가 인코딩된 벡터가 저장되어 있다.

그렇다면 최종적으로 이용하는 단어의 분산 표현으로는 어느 쪽 가중치를 선택하면 좋을까?

* 어떤 가중치를 선택하면 좋을까?
    * A. 입력 측의 가중치만 이용한다
    * B. 출력 측의 가중치만 이용한다.
    * C. 양쪽 가중치를 모두 이용한다.

word2vec(특히 skip-gram 모델)에서는 A안인 '입력 측의 가중치만 이용한다'가 가장 대중적인 선택이다.<br>
한편 word2vec과 비슷한 기법인 GloVe에서는 두 가중치를 더했을 때 좋은 결과를 얻었다. 